In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
data = Path("../data")
plot_path = data / "plots"
map_path = data / "graph3_map"
stat_path = map_path / "stats"
geojson_path = map_path / "geojson"

# Load geojson data


In [ ]:
import json

# Load custom.geo.json
with open(geojson_path / "custom.geo.json") as f:
    geojson = json.load(f)

In [ ]:
# Each of the feature in "features" jas a "properties" fields, and inside of this field an "admin" fields. Put all these in a dataframe
df_geojson = pd.DataFrame(
    [
        {
            "country": feature["properties"]["admin"],
        }
        for feature in geojson["features"]
    ]
)
print(*df_geojson["country"].sort_values().unique(), sep="\n")

Afghanistan
Albania
Algeria
Angola
Argentina
Armenia
Australia
Austria
Azerbaijan
Bangladesh
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Central African Republic
Chad
Chile
China
Colombia
Costa Rica
Croatia
Cuba
Cyprus
Czechia
Democratic Republic of the Congo
Denmark
Djibouti
Dominican Republic
East Timor
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands
Fiji
Finland
France
Gabon
Gambia
Georgia
Germany
Ghana
Greece
Greenland
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Israel
Italy
Ivory Coast
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kosovo
Kuwait
Kyrgyzstan
Laos
Latvia
Lebanon
Lesotho
Liberia
Libya
Lithuania
Luxembourg
Madagascar
Malawi
Malaysia
Mali
Mauritania
Mexico
Moldova
Mongolia
Montenegro
Morocco
Mozambique
Myanmar
Namibia
Nepal
Netherlands
New Caledonia
New Zealand
Nicaragua
Niger
Nigeria

In [ ]:
geojson = pd.read_json(data / "map/custom.geo.json")
geojson

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
...,...,...
174,FeatureCollection,"{'type': 'Feature', 'properties': {'featurecla..."
175,FeatureCollection,"{'type': 'Feature', 'properties': {'sovereignt..."
176,FeatureCollection,"{'type': 'Feature', 'properties': {'sovereignt..."
177,FeatureCollection,"{'type': 'Feature', 'properties': {'sovereignt..."
